## 알라딘 OpenAPI 가이드 
* (최종 수정일: 2022.7.13)

* 출처 : https://blog.aladin.co.kr/openapi/6695306
* [알라딘 Open API 매뉴얼 - Google Docs](https://docs.google.com/document/d/1mX-WxuoGs8Hy-QalhHcvuV17n50uGI2Sg_GHofgiePE/edit)



## OpenAPI 개요

OpenAPI는 크게 3가지로 나뉩니다:

1. 검색 API
2. 상품 API (단일 상품 조회)
3. 상품 리스트 API (베스트셀러 등 다양한 상품 리스트)

- 상품 API는 단일 상품 조회로, 검색 API보다 더 자세한 부가 정보를 제공합니다.
- 모든 API 결과는 XML(기본값) 또는 JSON 형식으로 제공됩니다.
- GET 및 POST 방식 모두 지원됩니다.
- API 사용은 일일 5,000회로 제한됩니다. (추가 사용은 별도 문의)

## 상품 리스트 API

### 제공 리스트 종류

- 신간 전체 리스트
- 주목할 만한 신간 리스트
- 편집자 추천 리스트 (카테고리별 조회 - 국내도서/음반/외서만 지원)
- 베스트셀러
- 북플 베스트셀러 (국내도서만 조회 가능)

### 요청 방법

- 요청 URL: `http://www.aladin.co.kr/ttb/api/ItemList.aspx`
- 요청 URL 샘플:
  ```
  http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey=TTBKey&QueryType=ItemNewAll&MaxResults=10&start=1&SearchTarget=Book&output=xml&Version=20131101
  ```

### 결과 샘플

- XML 형식: [http://www.aladin.co.kr/ttb/api/test/ItemList_20131101.xml](http://www.aladin.co.kr/ttb/api/test/ItemList_20131101.xml)
- JavaScript 형식: [http://www.aladin.co.kr/ttb/api/test/ItemList_20131101.js](http://www.aladin.co.kr/ttb/api/test/ItemList_20131101.js)

**참고**: 총 결과는 최대 1,000개까지만 조회 가능합니다.

* 터미널에서 환경변수 설정하기
    * Linux/Mac: export ALADIN_TTB_KEY=your_ttb_key_here
    * Windows: set ALADIN_TTB_KEY=your_ttb_key_here

In [ ]:
import os
# os.environ['ALADIN_TTB_KEY'] = "ttb-------"
os.environ.get('ALADIN_TTB_KEY')

In [ ]:
import requests
import xml.etree.ElementTree as ET
import sqlite3
import os
import time

category_dict = {
    8537: "컴퓨터공학/전산학 개론",
    32812: "그래픽/멀티미디어",
    27660: "프로그래밍 언어",
    28053: "네트워크",
    8482: "운영체제/소프트웨어 공학",
    8483: "자료구조",
    8484: "알고리즘",
    8485: "데이터베이스 개론",
    8486: "전산수학",
    8487: "정보통신 공학",
    8488: "컴퓨터 구조",
    8467: "컴퓨터공학"
}

def search_books(ttbkey, category_id, max_results=50, start=1):
    base_url = "http://www.aladin.co.kr/ttb/api/ItemList.aspx"
    
    params = {
        'TTBKey': ttbkey,
        'QueryType': 'ItemNewAll',
        'MaxResults': max_results,
        'start': start,
        'SearchTarget': 'Book',
        'output': 'xml',
        'Version': '20131101',
        'CategoryId': category_id
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        
        root = ET.fromstring(response.content)
        items = root.findall('.//item')
        
        if not items:
            return None
        
        books = []
        for item in items:
            book = {
                'title': item.find('title').text,
                'author': item.find('author').text,
                'isbn13': item.find('isbn13').text,
                'publisher': item.find('publisher').text,
                'pubdate': item.find('pubDate').text,
                'price': int(item.find('priceStandard').text),
                'description': item.find('description').text,
                'category_id': category_id
            }
            books.append(book)
        
        return books
    except requests.RequestException as e:
        print(f"API 요청 중 오류 발생: {e}")
        return None
    except ET.ParseError as e:
        print(f"XML 파싱 중 오류 발생: {e}")
        return None
    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")
        return None

def create_database():
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS books (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        author TEXT,
        isbn13 TEXT UNIQUE,
        publisher TEXT,
        pubdate TEXT,
        price INTEGER,
        description TEXT,
        category_id INTEGER
    )
    ''')
    conn.commit()
    return conn

def insert_books(conn, books):
    if not books:
        return
    
    cursor = conn.cursor()
    print(books[0]["title"])
    for book in books:
        try:
            print("insert : ", book['title'])
            cursor.execute('''
            INSERT OR IGNORE INTO books (title, author, isbn13, publisher, pubdate, price, description, category_id)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (book['title'], book['author'], book['isbn13'], book['publisher'], book['pubdate'], book['price'], book['description'], book['category_id']))
        except sqlite3.IntegrityError:
            pass  # 중복된 ISBN은 무시
    conn.commit()

def collect_books_by_category(ttbkey, category_ids):
    conn = create_database()
    
    for category_id in category_ids:
        total_books = 0
        start = 1
        
        while True:
            books = search_books(ttbkey, category_id, max_results=50, start=start)
            print("수집결과 :", category_dict[category_id], books)
            if not books:
                break
                
            print("books 0 : ", books[0]["title"])
            
            insert_books(conn, books)
            total_books += len(books)
            
            print(f"카테고리 {category_id}: {total_books}개의 도서 정보 수집 (시작 인덱스: {start})")
            
            if len(books) < 50:
                break
            
            start += 50
            time.sleep(1)  # API 요청 간 1초 대기
        
        print(f"카테고리 {category_id} 수집 완료: 총 {total_books}개의 도서")
    
    conn.close()

def verify_data():
    conn = sqlite3.connect('books.db')
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM books")
    count = cursor.fetchone()[0]
    
    print(f"\n데이터베이스의 총 도서 수: {count}")
    
    if count > 0:
        print("\n카테고리별 도서 수:")
        cursor.execute("SELECT category_id, COUNT(*) FROM books GROUP BY category_id")
        for category_id, category_count in cursor.fetchall():
            print(f"카테고리 {category_id}: {category_count}개")
        
        print("\n샘플 데이터:")
        cursor.execute("SELECT * FROM books LIMIT 5")
        books = cursor.fetchall()
        for book in books:
            print(f"제목: {book[1]}")
            print(f"저자: {book[2]}")
            print(f"ISBN-13: {book[3]}")
            print(f"출판사: {book[4]}")
            print(f"출판일: {book[5]}")
            print(f"가격: {book[6]}")
            print(f"카테고리 ID: {book[8]}")
            print(f"설명: {book[7][:100]}...")  # 설명의 처음 100자만 출력
            print('-' * 50)
    else:
        print("데이터베이스에 도서가 없습니다.")
    
    conn.close()

# 메인 실행 부분
if __name__ == "__main__":
    try:
        ttbkey = os.environ.get('ALADIN_TTB_KEY')
        if not ttbkey:
            raise ValueError("ALADIN_TTB_KEY 환경 변수가 설정되지 않았습니다.")
        
        category_ids = [8537, 32812, 27660, 28053, 8482, 8483, 8484, 8485, 8486, 8487, 8488, 8467]
        
        collect_books_by_category(ttbkey, category_ids)
        verify_data()
    except Exception as e:
        print(f"프로그램 실행 중 오류 발생: {e}")


In [ ]:
import sqlite3
import pandas as pd

def get_books_dataframe():
    # SQLite 데이터베이스에 연결
    conn = sqlite3.connect('books.db')
    
    # SQL 쿼리 실행 및 데이터프레임으로 변환
    query = "SELECT * FROM books"
    df = pd.read_sql_query(query, conn)
    
    # 데이터베이스 연결 종료
    conn.close()
    
    return df

def print_dataframe_info(df):
    print("데이터프레임 정보:")
    print(f"행 수: {df.shape[0]}")
    print(f"열 수: {df.shape[1]}")
    print("\n컬럼명:")
    print(df.columns.tolist())
    print("\n데이터 타입:")
    print(df.dtypes)
    display("\n처음 5개 행:")
    print(df.head())
    display("\n기본 통계 정보:")
    display(df.describe())

# 메인 실행 부분
if __name__ == "__main__":
    # 데이터프레임 가져오기
    books_df = get_books_dataframe()
    
    # 데이터프레임 정보 출력
    print_dataframe_info(books_df)